In [1]:
import requests
import time
import json
import numpy as np 
import random

DIRECTORY_NAME = "data/"


AUTH_TOKENS = [
    "03AKH6MRGfsxcnXXHz42b-rVmvIdqyihro1nLGOC7M7__kVGYhi_RrRWbmly-9Gw9WjVJBGcC56pO9phaU2J4N4d19wcdWMwEoMbsOUtjwOBAd5zMNvCgJ3lqCSBD5TjUQbceWZ9_NEhzPhdcvOwCQX43oWccNJlb8K-cONdhLUfz3ATAIo1DwJJ7xJ1-8zBWXEk8KSUV_kJv8Dc7X0jXHP-6stRkBqCy7vb_2N6iaBkZiN6arjmAv_dLAuEAociG-omwAHY-SObhs7q2SBGRHRs1FEW-J5CmEN6YSavK5XLH8TbfJ8pwhjNisNkk6B9eZTOSrvUj5FFVsdpm_JRz93-dm-RCKVuYY7vU32kitQpvLXHIJAEqttQKFBwmuo8I0DNDMpHhREUGfamPnB5a0aCeVzy4P7aLt4iSuZgCNKv-9mPMY7lYgvdBHxohxzKipY5izVfnTNLuHGPyEuFndiDhWcqv5LVt6HItwJibkWnOFiTStqW9kBSzjmkbBiuogzvZPdThH4LHxhqWm4i4LNNwnjfPe3ndyDw",
    "03AKH6MREVzaO5dbQnPk8EBNnsrA2f0oo9adZdp7d8Td3mHVUeFjWyyPEV10gwVK35RLN_F1HcK1yxp9x_TeFTeKh_l70QupyCLasw-jpyp2ggp172Q1Ky-oJvClOKJHaWd7k_qoPuNXkR4J7KbiRypgpafY0K19_g3TH8dGmvjukn8V4SAxSfCO58BDfFhIz-TbQCXx-Cw63MmnAt9QhjY9Vx9xY5KRVCdSpdovUjE77coT_9pbohALg_2gpwAJ9pttPg_UNqtaDAW9MGCFiLaFX5D-FCXNRsJ06_rVM8u6TPPdLhBTRhpC-d1BoG8ToEDCTERtxs3tj-sYW5TuuUsDmhKCDWyL9v41JGrE5duAQdZ6eX980cf6UZgj4m8SYPiE8yy3wgV42LM51YKA27KwIV_86qKNBSm1DqPDG_HvpZEUnMLzt9XVrknRyoXpfxV_rMxkeuhFDIFRaKmV5xIU6OhwUTcCjEHFn6OsReWFpZyLF7jbS66Ezap2IyZblebPZRMAXivGpvkBoqCO8Km3G41AbceyKz6g",
    "03AKH6MRGko8aHTZJr7d8VXTYsXf14ANRGCl7nv4yo2rviEzYvuvg63NYdbRBppj4hIiU_dDXpEbZJ5e57bDRgcw7VctOqnK_o6Q3v3eCFUvUea_76NE0DJLGnsByinwH7ot0b7mlbFN_mRo5I1cUf6_Li-s5Y1uHJPLrQlq27dhu-B_H3KAon2zFV8nwUKYBRfA6B55WIma6A-efHq9cUJCpz2SsRY4fx0iPDapKz1S3pXb6Lg-GR9etaGr24XDJzqunOLLonT6289fSR6PDmnOzlxfYuiE-kkh9nhrD39DFV7-nhzb1yOqqPeSkoCIFX9-Eehgbl4jT181AFS-uK_xr6gy-NcsAlUYqtzBAWyojg_bQQ5c_NUGdI2j5qg1AipBaZwH1Hy_C8vyh99ALNx0I2P_iL4k4GbiamhlTrYiRujBE679A-Y3ioHbdBSRvThO0Sj2bc7ftiusCmO-ffbV_o8OaLK6Vdk6HF6SMCA3Zalh-JZSu9av7WKC1LeTcM1HoKcOctamsLT4gr9i9N87j1Tmi0Db1ZD6EQAH_AZ7Elo9OkVe7TW10"
]

SLEEP_T = 2

# helper functions for making queries
postQueryEndpoint = "https://data.stackexchange.com/query/save/1"
getQueryEndpoint = "https://data.stackexchange.com/query/job/jobId"


def saveAsJson(fileName, content):
    # Serializing json
    json_object = json.dumps(content, indent=4)
 
    # Writing to sample.json
    with open(fileName, "w") as outfile:
        outfile.write(json_object)

global lastAuthToken
lastAuthToken = 0
def getAuthToken():
    global lastAuthToken
    lastAuthToken = (lastAuthToken + 1) % len(AUTH_TOKENS)
    return AUTH_TOKENS[lastAuthToken]

def makeQuery(query, fileName=None):
    query = {
        "sql": query,
        "g-recaptcha-response": getAuthToken()
    }

    # create initial query
    response = requests.post(postQueryEndpoint, data = query)
    response = response.json()

    running = response.get("running")
    getEndpoint = getQueryEndpoint

    # if this is a larger query, modify the get endpoint
    if running:
        getEndpoint = getEndpoint.replace("jobId", response.get("job_id"))

    # continue checking every interval for results
    while running:
        time.sleep(SLEEP_T)
        response = requests.get(getEndpoint, params = {"_": time.time()}).json()
        running = response.get("running")

    print(response)
    
    # save content to json if fileName provided
    content = response.get("resultSets")[0]
    if fileName: saveAsJson(fileName, content)

    return content

In [20]:
import pandas as pd

df = pd.read_csv("Posts.csv")

df.groupby('UserId').count()

# print(c)


,Id,PostHistoryTypeId,PostId,RevisionGUID,CreationDate,UserDisplayName,Comment,Text,ContentLicense
UserId,,,,,,,,,
10015258,5,5,5,5,5,0,1,5,5
10021446,3,3,3,3,3,0,0,3,3
1002843,6,6,6,6,6,0,0,6,6
10034619,37,37,37,37,37,0,4,37,37
10037094,1,1,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
9952809,3,3,3,3,3,0,0,3,3
9974792,36,36,36,36,36,0,5,36,36
9980148,30,30,30,30,30,0,8,30,30
